In [48]:
import os
from dotenv import load_dotenv
import pandas as pd
from sqlalchemy import create_engine



In [49]:
load_dotenv()

DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_ENTRADA = os.getenv("DB_ENTRADA")
DB_SAI = os.getenv("DB_SAI")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")

ENTRADA = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_ENTRADA}"
SAIDA = f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_SAI}'

engine_entrada = create_engine(ENTRADA)
engine_saida = create_engine(SAIDA)


query = "SELECT * FROM Analista"
dados_originais_df = pd.read_sql(query, engine_entrada)
df_anonimizado = dados_originais_df.copy()


In [50]:
# pseudonimizar nome
def pseudonimizar_nome(nome_completo):
    # percorre o tamanho do nome recebido (cada letra)
    for i in range(len(nome_completo)):
        # cria uma string com o texto "Analista" e o número da posição
        nome = str("Analista " + str(i))
        # retorna o nome (mas aqui ele para logo na primeira volta do loop)
        return nome_completo

#Faz com que o nome do analista seja impresso como "Analista 1", "Analista 2", etc
df_anonimizado["nome_completo"] = [
    f"Analista {i+1}" for i in range(len(df_anonimizado))
]
#Imprime a anonimização 
print("==== Dados anonimizados ====")
display(df_anonimizado[['nome_completo']])


==== Dados anonimizados ====


,nome_completo
0,Analista 1
1,Analista 2
2,Analista 3
3,Analista 4
4,Analista 5


In [16]:
import bcrypt

def criptografar(senha):
    
    salt = bcrypt.gensalt()
    hash_senha = bcrypt.hashpw(senha.encode('utf-8'), salt)
    return hash_senha.decode('utf-8')

def verificar_senha(senha_digitada, senha_criptografada):

    return bcrypt.checkpw(senha_digitada.encode('utf-8'), senha_criptografada.encode('utf-8'))


df_anonimizado["senha"] = df_anonimizado["senha"].apply(criptografar)

print("==== Dados anonimizados ====")
display(df_anonimizado[["senha"]])





==== Dados anonimizados ====


,senha
0,$2b$12$ydPlg8QU3HGeaVtBIquxL.SL5lA0qsGU.jyDq0F...
1,$2b$12$LEilXa.LoD9Nr4caX5Ib.uH0kdc4JGxUVPdxAAg...
2,$2b$12$NDcthC843RqDpLyQ/EuwdO3MzgtAd4CiygDteHq...
3,$2b$12$z6PwHf9fpdlbUtu8hRgNzu9lP5GsK7Wo4leiwiG...
4,$2b$12$vQRF5QGD2rv6DziAY5QGJeoh/MLvPyzppmlY61u...


In [51]:
# Pseudonimização da data de contratação
from IPython.display import display


def pseudonimizar_data(dt_contratacao):
   # Garante que é string para que possa remover espaços 
    data = str(dt_contratacao).strip()
    
    # Remove hífen e barra
    data = data.replace("-", "").replace("/", "")
    
    # Função para anonimizar 4 primeiros caracteres
    if len(data) > 4:
        return data[:4] + "****"
    else:
        return data + "****"


# Aplica e mostra a pseudonimização
df_anonimizado["dt_contratacao"] = df_anonimizado["dt_contratacao"].apply(pseudonimizar_data)

print("==== Dados anonimizados ====")
display(df_anonimizado[["dt_contratacao"]])


==== Dados anonimizados ====


,dt_contratacao
0,2023****
1,2023****
2,2023****
3,2023****
4,2023****


In [52]:
#Pseudonimização cargo
from IPython.display import display

def pseudonimizar(cargo):
    # Garante que é string para que possa remover espaços 
    palavra = str(cargo).strip()
    
    # Remove hífens e espaços
    palavra = palavra.replace("-", "").replace(" ", "")
    
    # Substitui a partir do 5º caractere por ****
    if len(palavra) > 1:
        valor_pseudo = palavra[:1] + "****"
    else:
        valor_pseudo = palavra + "****"
    
    return valor_pseudo


# Aplica pseudonimização na coluna "cargo"
df_anonimizado["cargo"] = df_anonimizado["cargo"].apply(pseudonimizar)

print("==== Dados anonimizados ====")
display(df_anonimizado[["cargo"]])



==== Dados anonimizados ====


,cargo
0,A****
1,A****
2,A****
3,C****
4,A****


In [53]:
# Anonimização analista 

from IPython.display import display


def anonimizar(senha, chave):
    senha = str(senha)
    chave = str(chave).lower()
    senha_cripto = ''
    i = 0
    for l in senha:
        deslocamento = ord(chave[i % (len(chave))]) - ord('a')
        nova_letra = chr((ord(l) - 32 + deslocamento) % 95 + 32)
        senha_cripto += nova_letra
        i += 1
    return senha_cripto

chave = "minhachave"
df_anonimizado["cpf"] = df_anonimizado["cpf"].apply(lambda x: anonimizar(x, chave))

print("==== Dados anonimizados ====")
display(df_anonimizado[["cpf"]])



==== Dados anonimizados ====


,cpf
0,=:@;58>8N5>
1,>;A<69?9E6?
2,?<B=7:@0F7@
3,@=C>8;71G8A
4,A>D?9282H9B


In [54]:
# Pseudonimização email
def pseudonimizar(email):
    email = str(email)
    parte_local, dominio = email.split("@")
  
    if len(parte_local) > 3:
        parte_local_pseudo = parte_local[:2] + "****"
    else:
        parte_local_pseudo = parte_local + "****"
    return parte_local_pseudo + "@" + dominio

df_anonimizado["email"] = df_anonimizado["email"].apply(pseudonimizar)

print("==== Dados anonimizados ====")
display(df_anonimizado[["email"]])


==== Dados anonimizados ====


,email
0,ra****@eitruck.com
1,ca****@eitruck.com
2,le****@eitruck.com
3,fe****@eitruck.com
4,ga****@eitruck.com


In [55]:
def pseudonimizar_telefone(telefone):
    s = str(telefone)
    if len(s) != 11 or not s.isdigit():
        return telefone  
    return "**" + s[2:5] + "****" + s[9:11]

df_anonimizado["telefone"] = df_anonimizado["telefone"].apply(pseudonimizar_telefone)


print("==== Dados anonimizados ====")
display(df_anonimizado[["telefone"]])


==== Dados anonimizados ====


,telefone
0,**987****21
1,**976****10
2,**965****09
3,**954****98
4,**943****87


In [56]:
# anonimizar id de unidade 
def pseudonimizar(texto):
    resultado = ""
    for caractere in str(texto):  # força string
        if caractere.isdigit():
            resultado += "*"
        else:
            resultado += caractere
    return resultado

df_anonimizado["id_unidade"] = df_anonimizado["id_unidade"].apply(pseudonimizar)

print("==== Dados anonimizados ====")
display(df_anonimizado[["id_unidade"]])



==== Dados anonimizados ====


,id_unidade
0,*
1,*
2,*
3,*
4,*


In [58]:
display(df_anonimizado)

,id,id_unidade,cpf,dt_contratacao,nome_completo,email,senha,cargo,telefone
0,1,*,=:@;58>8N5>,2023****,Analista 1,ra****@eitruck.com,60f6c3fdd8f2efc9aada81b663a5134f4b338b6fb80ad7...,A****,**987****21
1,2,*,>;A<69?9E6?,2023****,Analista 2,ca****@eitruck.com,ef92b778bafe771e89245b89ecbc08a44a4e166c066599...,A****,**976****10
2,3,*,?<B=7:@0F7@,2023****,Analista 3,le****@eitruck.com,daaad6e5604e8e17bd9f108d91e26afe6281dac8fda009...,A****,**965****09
3,4,*,@=C>8;71G8A,2023****,Analista 4,fe****@eitruck.com,a277c3ba4d39d7a49cc6507e1818e0d6ebc3d8621ccc63...,C****,**954****98
4,5,*,A>D?9282H9B,2023****,Analista 5,ga****@eitruck.com,240be518fabd2724ddb6f04eeb1da5967448d7e831c08c...,A****,**943****87


In [24]:
dados_originais_df.to_csv('original_analista.csv')
df_anonimizado.to_csv('pseudo_analista.csv')

In [59]:
df_anonimizado.to_sql(name='analista', con=engine_saida, if_exists='replace', index=False)

5